In [1]:
:set -XFlexibleContexts -XMonadComprehensions -XNoImplicitPrelude -XRebindableSyntax
import Language.Stochaskell
import System.Directory
setCurrentDirectory "stochaskell"
createDirectoryIfMissing True "cache/stan"

In [2]:
logreg :: Z -> Z -> P (RMat,RVec,R,BVec)
logreg n d = do
  x <- uniforms (matrix [ -10000 | i <- 1...n, j <- 1...d ])
                (matrix [  10000 | i <- 1...n, j <- 1...d ])
  w <- normals (vector [ 0 | j <- 1...d ]) (vector [ 3 | j <- 1...d ])
  b <- normal 0 3
  let z = (x #> w) + cast b
  y <- bernoulliLogits z
  return (x,w,b,y)

In [3]:
let n = 1000; d = 2
(xData,wTrue,bTrue,yData) <- simulate (logreg n d)
(wTrue,bTrue)

([0.907666841609812,1.720454663391033],-4.218830747430737)

In [4]:
samples <- hmcStan 250 [ (w,b) | (x,w,b,y) <- logreg n d, x == xData, y == yData ]

--- Generating Stan code ---
data {
  matrix[1000,2] x_stan_0_0;
  
  
  int<lower=0,upper=1> x_stan_0_3[1000];
}
parameters {
  
  vector[2] x_stan_0_1;
  real x_stan_0_2;
  
}
model {
  
  
  
  vector[1000] v_0_3;
  vector[1000] v_0_4;
  vector[2] v_0_5;
  vector[2] v_0_6;
  
  
  
  
  
  v_0_3 = to_matrix(x_stan_0_0) * to_vector(x_stan_0_1);
  v_0_4 = x_stan_0_2 + v_0_3;
  for (i_1_1 in 1:2) {
  
    v_0_5[i_1_1] = 0;
  }
  for (i_1_1 in 1:2) {
  
    v_0_6[i_1_1] = 3;
  }
  
  

  
  x_stan_0_1 ~ normal(v_0_5, v_0_6);
  x_stan_0_2 ~ normal(0, 3);
  x_stan_0_3 ~ bernoulli_logit(v_0_4);
}

make -C /home/jovyan/stochaskell/cmdstan /home/jovyan/stochaskell/cache/stan/model_d5e39b725ce29e2903feda152dcf4aa3601a40d5
make: Entering directory '/home/jovyan/stochaskell/cmdstan'

--- Translating Stan model to C++ code ---
bin/stanc  /home/jovyan/stochaskell/cache/stan/model_d5e39b725ce29e2903feda152dcf4aa3601a40d5.stan --o=/home/jovyan/stochaskell/cache/stan/model_d5e39b725ce29e2903feda152d

In [5]:
let (ws,bs) = unzip samples
(mean ws, mean bs)

([1.0427378640000002,1.976941208],-0.8679887320000002)